In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('kddcup99_csv.csv')

In [3]:
df.shape

(494020, 42)

In [4]:
num_feature = ['duration',
 'protocol_type',
 'src_bytes',
 'dst_bytes',
 'wrong_fragment',
 'urgent',
 'num_failed_logins',
 'lnum_compromised',
 'lroot_shell',
 'lsu_attempted',
 'lnum_file_creations',
 'lnum_shells',
 'lnum_access_files',
 'lnum_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'dst_host_count',
 'dst_host_srv_count',
              'label']

In [5]:
data = df[num_feature]

In [6]:
data['label'].unique()

array(['normal', 'buffer_overflow', 'loadmodule', 'perl', 'neptune',
       'smurf', 'guess_passwd', 'pod', 'teardrop', 'portsweep', 'ipsweep',
       'land', 'ftp_write', 'back', 'imap', 'satan', 'phf', 'nmap',
       'multihop', 'warezmaster', 'warezclient', 'spy', 'rootkit'],
      dtype=object)

In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['protocol_type'] = le.fit_transform(data['protocol_type'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
a = []
for i in range(data.shape[0]):
    if str(data['label'][i]) not in ['normal', 'smurf', 'neptune']:
        a.append(i)
data = data.drop(data.index[[a]])

In [9]:
data.reset_index(drop=True, inplace=True)

In [10]:
data.shape

(485268, 19)

In [11]:
# from sklearn.model_selection import train_test_split
# kdd99_train, kdd99_test = train_test_split(data, test_size=0.2, random_state = 42)

In [12]:
# kdd99_train_csv = kdd99_train.to_csv ('kdd99_train.csv', index = None, header=True)
# kdd99_test_csv = kdd99_test.to_csv ('kdd99_test.csv', index = None, header=True)

In [13]:
X= data.iloc[:,0:18]
y= data.iloc[:,18]

In [14]:
X.head(2)

,duration,protocol_type,src_bytes,dst_bytes,wrong_fragment,urgent,num_failed_logins,lnum_compromised,lroot_shell,lsu_attempted,lnum_file_creations,lnum_shells,lnum_access_files,lnum_outbound_cmds,is_host_login,is_guest_login,dst_host_count,dst_host_srv_count
0,0,1,181,5450,0,0,0,0,0,0,0,0,0,0,0,0,9,9
1,0,1,239,486,0,0,0,0,0,0,0,0,0,0,0,0,19,19


In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.5/dist-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [16]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)

Using TensorFlow backend.


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.2)

In [18]:
from keras import Sequential
from keras.layers import Dense

In [19]:
classifier = Sequential()
#First Hidden Layer
classifier.add(Dense(60, activation='relu', kernel_initializer='random_normal', input_dim=18))
#Second  Hidden Layer
classifier.add(Dense(60, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(60, activation='relu', kernel_initializer='random_normal'))
#Output Layer
classifier.add(Dense(3, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [20]:
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [21]:
classifier.fit(X_train,y_train, batch_size=10, epochs=5)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
388214/388214 [==============================] - 41s 107us/step - loss: 0.0059 - acc: 0.9987
Epoch 2/5
388214/388214 [==============================] - 41s 105us/step - loss: 0.0035 - acc: 0.9993
Epoch 3/5
388214/388214 [==============================] - 43s 110us/step - loss: 0.0035 - acc: 0.9994
Epoch 4/5
388214/388214 [==============================] - 42s 109us/step - loss: 0.0035 - acc: 0.9994
Epoch 5/5
388214/388214 [==============================] - 44s 113us/step - loss: 0.0030 - acc: 0.9994


In [22]:
from keras.models import model_from_json
# serialize model to JSON
model_json = classifier.to_json()
with open("model_ANN.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
classifier.save_weights("model_ANN.h5")
print("Saved model to disk")

Saved model to disk


In [23]:
json_file = open('model_ANN.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_ANN.h5")
print("Loaded model from disk")

Loaded model from disk


In [24]:
loaded_model.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [25]:
y_pred=loaded_model.predict(X_test)
y_pred =(y_pred>0.5)

In [26]:
test_loss, test_acc = loaded_model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

97054/97054 [==============================] - 1s 14us/step
test_acc:  0.9995191679119358


In [27]:
from sklearn.metrics import confusion_matrix
labels = [0, 1, 2]
le_name_mapping = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))
print(le_name_mapping)
cmtx = pd.DataFrame(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1), labels=labels), index=['neptune','normal','smurf'], columns=['neptune','normal','smurf'])
print(cmtx)

{'normal': 1, 'neptune': 0, 'smurf': 2}
         neptune  normal  smurf
neptune    21300      47      0
normal        21   19461      0
smurf          0       2  56223
